### Download VOC 2007 and save it to locally

In [16]:
import os

project_path = os.getcwd()
root_dir = '/content/data'
data_exist = False

try:
  os.mkdir('/content')
  os.mkdir(root_dir) 
  os.chdir(root_dir) 
except:
  data_exist = True

In [17]:
if not data_exist:
  !curl -LO "https://pjreddie.com/media/files/VOCtrainval_11-May-2012.tar"
  !curl -LO "https://pjreddie.com/media/files/VOCtrainval_06-Nov-2007.tar"
  !curl -LO "https://pjreddie.com/media/files/VOCtest_06-Nov-2007.tar"
  !tar -xvf "VOCtrainval_06-Nov-2007.tar"
  !tar -xvf "VOCtest_06-Nov-2007.tar"
  !tar -xvf "VOCtrainval_11-May-2012.tar"
  !rm "VOCtrainval_06-Nov-2007.tar"
  !rm "VOCtest_06-Nov-2007.tar"
  !rm "VOCtrainval_11-May-2012.tar"  
  !sudo mv /content/data/VOCdevkit/VOC2012/JPEGImages/ /content/data/VOCdevkit/VOC2012/allimages/
#   !sudo mv /content/data/VOCdevkit/VOC2007/JPEGImages /content/data/VOCdevkit/VOC2012/allimages/
  !cp /content/data/VOCdevkit/VOC2007/JPEGImages/* /content/data/VOCdevkit/VOC2012/allimages/
  !rm /content/data/VOCdevkit/VOC2007/JPEGImages/*.jpg

os.chdir(project_path)

In [29]:
import datetime

now = datetime.datetime.now()
print(now)

2019-05-06 19:43:20.556345


In [26]:
!git status

On branch modified_yolo
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   dataset.py
	modified:   resnet_yolo.py
	modified:   train_yolo.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)

	best_prev.pth
	eval_loc.ipynb
	multi_loss.py
	temp.csv
	test_loss.ipynb
	train_log.csv
	visualize_logs.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models
from torch.autograd import Variable

from net import vgg16, vgg16_bn
from resnet_yolo import resnet50, resnet18
from yoloLoss import YoloLoss
from multi_loss import MultiLoss
from dataset import yoloDataset

from visualizer import Visualizer
import time
import numpy as np

In [19]:
use_local = os.path.exists('/home/stefan/data/VOCdevkit/VOC2007')

file_root = '/home/stefan/data/VOCdevkit/VOC2007/JPEGImages' if use_local else root_dir +  '/VOCdevkit/VOC2012/allimages/'

learning_rate = 0.001
num_epochs = 50
batch_size = 1
use_resnet = True
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
if use_resnet:
    net = resnet18()
else:
    net = vgg16_bn()


In [20]:
print('load pre-trined model')
if use_resnet:
    resnet = models.resnet18(pretrained=True)
    new_state_dict = resnet.state_dict()
    dd = net.state_dict()
    for k in new_state_dict.keys():
        print(k)
        if k in dd.keys() and not k.startswith('fc'):
            print('yes')
            dd[k] = new_state_dict[k]
    net.load_state_dict(dd)
else:
    vgg = models.vgg16_bn(pretrained=True)
    new_state_dict = vgg.state_dict()
    dd = net.state_dict()
    for k in new_state_dict.keys():
        print(k)
        if k in dd.keys() and k.startswith('features'):
            print('yes')
            dd[k] = new_state_dict[k]
    net.load_state_dict(dd)


load pre-trined model
conv1.weight
yes
bn1.weight
yes
bn1.bias
yes
bn1.running_mean
yes
bn1.running_var
yes
bn1.num_batches_tracked
yes
layer1.0.conv1.weight
yes
layer1.0.bn1.weight
yes
layer1.0.bn1.bias
yes
layer1.0.bn1.running_mean
yes
layer1.0.bn1.running_var
yes
layer1.0.bn1.num_batches_tracked
yes
layer1.0.conv2.weight
yes
layer1.0.bn2.weight
yes
layer1.0.bn2.bias
yes
layer1.0.bn2.running_mean
yes
layer1.0.bn2.running_var
yes
layer1.0.bn2.num_batches_tracked
yes
layer1.1.conv1.weight
yes
layer1.1.bn1.weight
yes
layer1.1.bn1.bias
yes
layer1.1.bn1.running_mean
yes
layer1.1.bn1.running_var
yes
layer1.1.bn1.num_batches_tracked
yes
layer1.1.conv2.weight
yes
layer1.1.bn2.weight
yes
layer1.1.bn2.bias
yes
layer1.1.bn2.running_mean
yes
layer1.1.bn2.running_var
yes
layer1.1.bn2.num_batches_tracked
yes
layer2.0.conv1.weight
yes
layer2.0.bn1.weight
yes
layer2.0.bn1.bias
yes
layer2.0.bn1.running_mean
yes
layer2.0.bn1.running_var
yes
layer2.0.bn1.num_batches_tracked
yes
layer2.0.conv2.weight
ye

In [21]:
# criterion = YoloLoss(5,0.5)
criterion = MultiLoss(5,0.5)

net = net.to(device)
net.train()
# different learning rate
params=[]
params_dict = dict(net.named_parameters())
for key,value in params_dict.items():
    if key.startswith('features'):
        params += [{'params':[value],'lr':learning_rate*1}]
    else:
        params += [{'params':[value],'lr':learning_rate}]
optimizer = torch.optim.SGD(params, lr=learning_rate, momentum=0.9, weight_decay=5e-4)

### Data loading

In [22]:
train_dataset = yoloDataset(root=file_root,list_file=os.path.join(project_path, 'voc2007.txt'),train=True,transform = [transforms.ToTensor()] )
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True,num_workers=4)
# test_dataset = yoloDataset(root=file_root,list_file='voc07_test.txt',train=False,transform = [transforms.ToTensor()] )
test_dataset = yoloDataset(root=file_root,list_file=os.path.join(project_path, 'voc2007test.txt'),train=False,transform = [transforms.ToTensor()] )
test_loader = DataLoader(test_dataset,batch_size=batch_size,shuffle=False,num_workers=4)
print('the dataset has %d images' % (len(train_dataset)))
print('the batch_size is %d' % (batch_size))

data init
data init
the dataset has 5012 images
the batch_size is 1


### Train loop

In [23]:
timer = time.time()
num_iter = 0
vis = Visualizer()
best_test_loss = np.inf

for epoch in range(num_epochs):
    net.train()
    if epoch == 1:
        learning_rate = 0.0005
    if epoch == 2:
        learning_rate = 0.00075
    if epoch == 3:
        learning_rate = 0.001
    if epoch == 30:
        learning_rate=0.0001
    if epoch == 40:
        learning_rate=0.00001

    for param_group in optimizer.param_groups:
        param_group['lr'] = learning_rate
    
    print('\n\nStarting epoch %d / %d' % (epoch + 1, num_epochs))
    print('Learning Rate for this epoch: {}'.format(learning_rate))
    
    total_loss = 0.
    
    for i,(images,target) in enumerate(train_loader):
        images = images.to(device)
        target = target.to(device)
        pred = net(images)
        loss = criterion(pred,target)
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i+1) % 5 == 0:
            cur_time = time.time()
            print ('Epoch [%d/%d], Iter [%d/%d] Loss: %.4f, average_loss: %.4f, time(s): %.2f' 
            %(epoch+1, num_epochs, i+1, len(train_loader), loss.item(), total_loss / (i+1), cur_time-timer))
            num_iter += 1
            timer = cur_time
            vis.add_log(("train_avrg_loss", total_loss/(i+1)))
    vis.add_log(("train_loss", total_loss / len(train_loader)), epoch)
    
    #validation
    validation_loss = 0.0
    net.eval()
    for i,(images,target) in enumerate(test_loader):
        images = images.to(device)
        target = target.to(device)     
        with torch.set_grad_enabled(False):
          pred = net(images)
          loss = criterion(pred,target)
          validation_loss += loss.item()
    validation_loss /= len(test_loader)
    vis.add_log(("val_loss", validation_loss), epoch)
    vis.save(os.path.join(project_path, 'temp.csv'))
    if best_test_loss > validation_loss:
        best_test_loss = validation_loss
        print('get best test loss %.5f' % best_test_loss)
        torch.save(net.state_dict(),'best.pth')
        

vis.save('train_log{}.csv'.format(time.time()))
torch.save(net.state_dict(),os.path.join(project_path, 'yolo.pth'))
vis.plot()





Starting epoch 1 / 50
Learning Rate for this epoch: 0.001


/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/site-packages/torch/nn/functional.py:1006: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


Epoch [1/50], Iter [5/5012] Loss: 58.7172, average_loss: 62.1232, time(s): 8.04
Epoch [1/50], Iter [10/5012] Loss: 39.0382, average_loss: 55.4229, time(s): 6.55


Process Process-9:
Process Process-10:
Process Process-11:
Traceback (most recent call last):
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Process Process-12:
Traceback (most recent call last):
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/stefan/programs/anaconda3/envs/ssd

KeyboardInterrupt: 

  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/multiprocessing/connection.py", line 911, in wait
    ready = selector.select(timeout)
  File "/home/stefan/programs/anaconda3/envs/ssd/lib/python3.6/selectors.py", line 376, in select
    fd_event_list = self._poll.poll(timeout)
KeyboardInterrupt
